In [ ]:
using Revise
@time using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Info);

In [ ]:
save_folder = <enter your save folder here could be tmp> # change to your save_folder

if !isdir(save_folder)
    mkdir(save_folder)
end

In [ ]:
old_dir = pwd()
mkpath(save_folder)
try
    cd(save_folder) # this is to save temporary distributed files in working folder
    FUSE.parallel_environment("localhost",5)
finally
    cd(old_dir)
end
display(pwd())


In [ ]:
#minimal working example for loading and saving dd using pmap
using Distributed
@everywhere import IMAS
@everywhere import FUSE

@everywhere function load_case(input_folder)
    IMAS.json2imas(joinpath(input_folder,"dd.json")),FUSE.json2act(joinpath(input_folder,"act.json")) 
end
   

@everywhere function run_case_try(folder,input_folder)
    if !isdir(folder)
        mkdir(folder)
    end
    
    try
        dd,act = load_case(input_folder)
        
        FUSE.ActorNoOperation(dd,act)
    
        FUSE.save(FUSE.memtrace, joinpath(folder, "memtrace.txt"))
        IMAS.imas2json(dd,joinpath(folder,"outputdd.json"))
    catch e
        println(e)
    end
    return nothing
end

@everywhere function run_case_no_try(folder,input_folder)

    if !isdir(folder)
        mkdir(folder)
    end
    
    dd,act = load_case(input_folder)

    FUSE.ActorNoOperation(dd,act)

    FUSE.save(FUSE.memtrace, joinpath(folder, "memtrace.txt"))
    IMAS.imas2json(dd,joinpath(folder,"outputdd.json"))

    return nothing
end


@everywhere function _run_case_no_try(folder,input_folder)
    tmp = run_case_no_try(folder,input_folder)
    tmp = nothing
    Gc.gc()
end


In [ ]:
N_runs = 5000
to_run = [joinpath(save_folder,"$i") for i in 1000:1:1000 + N_runs]
input_folder = save_folder

run_case_no_try(to_run[1],input_folder); # test 1 case

In [ ]:
pmap(folder -> run_case_no_try(folder, input_folder), to_run)    # load & save without gc
#pmap(folder -> _run_case_no_try(folder, input_folder), to_run)  # warpped function with gc

In [ ]:
# to see memory output

f1 = readdir(save_folder)[end-3]
@show f1
mem = FUSE.load(FUSE.MemTrace(),joinpath(save_folder,f1,"memtrace.txt"))

plot(mem,0)